In [10]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import sklearn
import statistics
import ray
from ray import tune
from ray.tune.schedulers import ASHAScheduler
import time

In [11]:
start_time = time.time()
train_data_frame = pd.read_csv('train.csv')
train_data_frame.head()

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,1,2596,51,3,258,0,510,221,232,148,...,0,0,0,0,0,0,0,0,0,5
1,2,2590,56,2,212,-6,390,220,235,151,...,0,0,0,0,0,0,0,0,0,5
2,3,2804,139,9,268,65,3180,234,238,135,...,0,0,0,0,0,0,0,0,0,2
3,4,2785,155,18,242,118,3090,238,238,122,...,0,0,0,0,0,0,0,0,0,2
4,5,2595,45,2,153,-1,391,220,234,150,...,0,0,0,0,0,0,0,0,0,5


In [12]:
train_Features = train_data_frame.iloc[:, :-1] 
# drop last column
train_Classes = train_data_frame.iloc[:, -1] 
# drop last column
RFclassifier = RandomForestClassifier()
params = RFclassifier.get_params()
print(params)

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


In [13]:
sklearn.model_selection.cross_validate(RFclassifier, train_Features, train_Classes)


{'fit_time': array([2.04295707, 1.9722681 , 1.98626828, 1.94304752, 1.95544243]),
 'score_time': array([0.06476426, 0.07664871, 0.07947087, 0.07705379, 0.07574081]),
 'test_score': array([0.45039683, 0.61177249, 0.69675926, 0.73578042, 0.7582672 ])}

In [ ]:
search_space = {
    "max_depth": tune.grid_search([5*i for i in range(1,10)]),
    "n_estimators": tune.grid_search([10,100,200,300]),
    "ccp_alpha": tune.grid_search([0.0,0.1,0.2]),
}
ray.init(address='auto')

def train(config):
    RFclassifier = RandomForestClassifier(**config)
    params = RFclassifier.get_params()
    xval = sklearn.model_selection.cross_validate(RFclassifier, train_Features, train_Classes)
    tune.report(mean_accuracy=statistics.mean(xval["test_score"]))

analysis = tune.run(train, config=search_space)



2022-05-12 16:18:37,406	INFO worker.py:946 -- Connecting to existing Ray cluster at address: 192.168.2.133:6379
(raylet) /usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
(raylet) /usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2022-05-12 16:18:40,277	INFO trial_runner.py:803 -- starting train_6a6b2_00000
(raylet) /usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
(raylet) /usr/lib/python3/dist-packages/requests/__init__.py:89: Reques

Trial name,status,loc,ccp_alpha,max_depth,n_estimators
train_6a6b2_00000,RUNNING,192.168.2.133:7366,0,5,10
train_6a6b2_00001,PENDING,,0.1,5,10
train_6a6b2_00002,PENDING,,0.2,5,10
train_6a6b2_00003,PENDING,,0,10,10
train_6a6b2_00004,PENDING,,0.1,10,10
train_6a6b2_00005,PENDING,,0.2,10,10
train_6a6b2_00006,PENDING,,0,15,10
train_6a6b2_00007,PENDING,,0.1,15,10
train_6a6b2_00008,PENDING,,0.2,15,10
train_6a6b2_00009,PENDING,,0,20,10


2022-05-12 16:18:43,367	INFO trial_runner.py:803 -- starting train_6a6b2_00001


Result for train_6a6b2_00000:
  date: 2022-05-12_16-18-43
  done: false
  experiment_id: f24c85ae8a1e4be78288926fa6cacd58
  hostname: patrick-c3-1
  iterations_since_restore: 1
  mean_accuracy: 0.6097222222222223
  node_ip: 192.168.2.133
  pid: 7366
  time_since_restore: 0.6023128032684326
  time_this_iter_s: 0.6023128032684326
  time_total_s: 0.6023128032684326
  timestamp: 1652365123
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 6a6b2_00000
  warmup_time: 0.004564523696899414
  
Result for train_6a6b2_00000:
  date: 2022-05-12_16-18-43
  done: true
  experiment_id: f24c85ae8a1e4be78288926fa6cacd58
  experiment_tag: 0_ccp_alpha=0.0,max_depth=5,n_estimators=10
  hostname: patrick-c3-1
  iterations_since_restore: 1
  mean_accuracy: 0.6097222222222223
  node_ip: 192.168.2.133
  pid: 7366
  time_since_restore: 0.6023128032684326
  time_this_iter_s: 0.6023128032684326
  time_total_s: 0.6023128032684326
  timestamp: 1652365123
  timesteps_since_restore: 0
  training_itera

2022-05-12 16:18:44,240	INFO trial_runner.py:803 -- starting train_6a6b2_00002
(raylet) /usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
(raylet) /usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


...

(raylet) /usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2022-05-12 16:39:36,593	INFO trial_runner.py:803 -- starting train_6a6b2_00106
(raylet) /usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Trial name,status,loc,ccp_alpha,max_depth,n_estimators,acc,iter,total time (s)
train_6a6b2_00105,RUNNING,192.168.2.133:13864,0,45,300,,,
train_6a6b2_00106,RUNNING,192.168.2.133:13932,0.1,45,300,,,
train_6a6b2_00107,PENDING,,0.2,45,300,,,
train_6a6b2_00000,TERMINATED,192.168.2.133:7366,0,5,10,0.609722,1,0.602313
train_6a6b2_00001,TERMINATED,192.168.2.133:7427,0.1,5,10,0.30496,1,0.551432
train_6a6b2_00002,TERMINATED,192.168.2.133:7432,0.2,5,10,0.142857,1,0.500516
train_6a6b2_00003,TERMINATED,192.168.2.133:7523,0,10,10,0.617725,1,0.863647
train_6a6b2_00004,TERMINATED,192.168.2.133:7525,0.1,10,10,0.24881,1,0.862069
train_6a6b2_00005,TERMINATED,192.168.2.133:7653,0.2,10,10,0.142857,1,0.820979
train_6a6b2_00006,TERMINATED,192.168.2.133:7656,0,15,10,0.624735,1,1.09692


Trial name,status,loc,ccp_alpha,max_depth,n_estimators,acc,iter,total time (s)
train_6a6b2_00105,RUNNING,192.168.2.133:13864,0,45,300,,,
train_6a6b2_00106,RUNNING,192.168.2.133:13932,0.1,45,300,,,
train_6a6b2_00107,PENDING,,0.2,45,300,,,
train_6a6b2_00000,TERMINATED,192.168.2.133:7366,0,5,10,0.609722,1,0.602313
train_6a6b2_00001,TERMINATED,192.168.2.133:7427,0.1,5,10,0.30496,1,0.551432
train_6a6b2_00002,TERMINATED,192.168.2.133:7432,0.2,5,10,0.142857,1,0.500516
train_6a6b2_00003,TERMINATED,192.168.2.133:7523,0,10,10,0.617725,1,0.863647
train_6a6b2_00004,TERMINATED,192.168.2.133:7525,0.1,10,10,0.24881,1,0.862069
train_6a6b2_00005,TERMINATED,192.168.2.133:7653,0.2,10,10,0.142857,1,0.820979
train_6a6b2_00006,TERMINATED,192.168.2.133:7656,0,15,10,0.624735,1,1.09692


Result for train_6a6b2_00105:
  date: 2022-05-12_16-39-47
  done: false
  experiment_id: 74e8acbcb27747d6b96cc03ede91185e
  hostname: patrick-c3-1
  iterations_since_restore: 1
  mean_accuracy: 0.6558862433862434
  node_ip: 192.168.2.133
  pid: 13864
  time_since_restore: 32.93022418022156
  time_this_iter_s: 32.93022418022156
  time_total_s: 32.93022418022156
  timestamp: 1652366387
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 6a6b2_00105
  warmup_time: 0.00528407096862793
  
Result for train_6a6b2_00105:
  date: 2022-05-12_16-39-47
  done: true
  experiment_id: 74e8acbcb27747d6b96cc03ede91185e
  experiment_tag: 105_ccp_alpha=0.0,max_depth=45,n_estimators=300
  hostname: patrick-c3-1
  iterations_since_restore: 1
  mean_accuracy: 0.6558862433862434
  node_ip: 192.168.2.133
  pid: 13864
  time_since_restore: 32.93022418022156
  time_this_iter_s: 32.93022418022156
  time_total_s: 32.93022418022156
  timestamp: 1652366387
  timesteps_since_restore: 0
  training_iterat

(raylet) /usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
(raylet)   warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2022-05-12 16:39:48,607	INFO trial_runner.py:803 -- starting train_6a6b2_00107


In [ ]:
stop_time = time.time()
print("Stopping at :", stop_time)
print("Total elapsed time: ", stop_time - start_time)

In [ ]:
ray.shutdown()